In [1]:
import lasio
from class2 import facies_classification, Petrophysics

In [2]:
las = lasio.read('data/Freeman-004-ST1.las')
well1 = las.df()
well = well1.reset_index()

In [3]:
well.head()

,DEPT,DEN,DT,GR,NEUT,RES_DEP
0,7564.5,2.2382,137.507,NaN,NaN,NaN
1,7565.0,2.2382,137.507,NaN,NaN,NaN
2,7565.5,2.2382,137.507,NaN,NaN,NaN
3,7566.0,2.2382,137.507,NaN,NaN,NaN
4,7566.5,2.2382,137.507,NaN,NaN,NaN


In [4]:
well['Facies'] = facies_classification(well['GR'])

In [5]:
well.head()

,DEPT,DEN,DT,GR,NEUT,RES_DEP,Facies
0,7564.5,2.2382,137.507,NaN,NaN,NaN,none
1,7565.0,2.2382,137.507,NaN,NaN,NaN,none
2,7565.5,2.2382,137.507,NaN,NaN,NaN,none
3,7566.0,2.2382,137.507,NaN,NaN,NaN,none
4,7566.5,2.2382,137.507,NaN,NaN,NaN,none


In [6]:
well['Facies'].unique()

array(['none', 'sand', 'shaly sand', 'shale'], dtype=object)

In [7]:
test = Petrophysics(well)

In [8]:
x = test.slowness_to_velocity(2)

In [9]:
well.head()

,DEPT,DEN,DT,GR,NEUT,RES_DEP,Facies,velocity
0,7564.5,2.2382,137.507,NaN,NaN,NaN,none,7272.357044
1,7565.0,2.2382,137.507,NaN,NaN,NaN,none,7272.357044
2,7565.5,2.2382,137.507,NaN,NaN,NaN,none,7272.357044
3,7566.0,2.2382,137.507,NaN,NaN,NaN,none,7272.357044
4,7566.5,2.2382,137.507,NaN,NaN,NaN,none,7272.357044


In [10]:
well.describe()

,DEPT,DEN,DT,GR,NEUT,RES_DEP,velocity
count,7586.000000,7586.000000,7586.000000,7352.000000,7352.000000,7350.000000,7586.000000
mean,9460.750000,2.326999,118.441103,108.189529,0.423099,18.933387,8527.682849
std,1095.016952,0.081725,11.924260,23.541196,0.066108,176.943528,844.760010
min,7564.500000,1.863100,84.811200,32.593900,0.099500,0.081200,6307.620110
25%,8512.625000,2.272600,108.603500,101.771775,0.385175,0.878350,7762.524203
50%,9460.750000,2.342500,115.442700,112.463100,0.422200,1.025600,8662.306062
75%,10408.875000,2.390700,128.824075,120.219250,0.461400,1.368400,9207.806378
max,11357.000000,2.564600,158.538400,219.338000,0.656900,1950.000000,11790.895542


In [12]:
well.to_csv('data/freeman_well_4.csv', index=False)

In [27]:
import pandas as pd
df = pd.read_csv('data/freeman_well_4_perm.csv')
df.head()

,DEPT,DEN,DT,GR,NEUT,RES_DEP,PHI,PERM
0,7564.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364
1,7565.0,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364
2,7565.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364
3,7566.0,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364
4,7566.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364


In [28]:
from class2 import facies_classification, Petrophysics
df['Facies'] = facies_classification(well['GR'])
df

,DEPT,DEN,DT,GR,NEUT,RES_DEP,PHI,PERM,Facies
0,7564.5,2.2382,137.5070,NaN,NaN,NaN,0.2657,175.29364,none
1,7565.0,2.2382,137.5070,NaN,NaN,NaN,0.2657,175.29364,none
2,7565.5,2.2382,137.5070,NaN,NaN,NaN,0.2657,175.29364,none
3,7566.0,2.2382,137.5070,NaN,NaN,NaN,0.2657,175.29364,none
4,7566.5,2.2382,137.5070,NaN,NaN,NaN,0.2657,175.29364,none
...,...,...,...,...,...,...,...,...,...
7581,11355.0,2.3986,111.3281,162.1151,0.551,0.9417,0.1622,1.49274,shale
7582,11355.5,2.3986,111.3281,162.1151,0.551,0.9418,0.1622,1.49274,shale
7583,11356.0,2.3986,111.3281,162.1151,0.551,0.9418,0.1622,1.49274,shale
7584,11356.5,2.3986,111.3281,162.1151,0.551,0.9419,0.1622,1.49274,shale


In [29]:
test = Petrophysics(df)
x = test.slowness_to_velocity(2)

In [30]:
x.head()

,DEPT,DEN,DT,GR,NEUT,RES_DEP,PHI,PERM,Facies,velocity
0,7564.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
1,7565.0,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
2,7565.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
3,7566.0,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
4,7566.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044


In [31]:
x.to_csv('data/well_4_ML.csv', index=False)

In [32]:
data = pd.read_csv('data/well_4_ML.csv')

In [33]:
data.head()

,DEPT,DEN,DT,GR,NEUT,RES_DEP,PHI,PERM,Facies,velocity
0,7564.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
1,7565.0,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
2,7565.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
3,7566.0,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
4,7566.5,2.2382,137.507,NaN,NaN,NaN,0.2657,175.29364,none,7272.357044
